<a href="https://colab.research.google.com/github/RidmaTabassum/Assignment-1/blob/main/Answer%208.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# 2.1
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

# Define LeNet with batch normalization
class LeNetBN(nn.Module):
    def __init__(self):
        super(LeNetBN, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.bn1 = nn.BatchNorm2d(6)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.bn2 = nn.BatchNorm2d(16)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.bn3 = nn.BatchNorm1d(120)
        self.fc2 = nn.Linear(120, 84)
        self.bn4 = nn.BatchNorm1d(84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool1(torch.relu(self.bn1(self.conv1(x))))
        x = self.pool2(torch.relu(self.bn2(self.conv2(x))))
        x = x.view(-1, 16*5*5)
        x = torch.relu(self.bn3(self.fc1(x)))
        x = torch.relu(self.bn4(self.fc2(x)))
        x = self.fc3(x)
        return x

# Define LeNet without batch normalization
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool1(torch.relu(self.conv1(x)))
        x = self.pool2(torch.relu(self.conv2(x)))
        x = x.view(-1, 16*5*5)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Define data transforms
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.2010])
])

# Load CIFAR-10 dataset
trainset = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
testset = datasets.FashionMNIST(root='./data', train=False, download=True,transform=transform )
# Load CIFAR-10 dataset
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
testset = datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

# Define data loaders
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=False, num_workers=2)

# Define device to train on
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Initialize LeNet models
net_bn = LeNetBN().to(device)
net = LeNet().to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer_bn = optim.SGD(net_bn.parameters(), lr=0.01, momentum=0.9)
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

# Train models for 10 epochs
for epoch in range(10):

    # Train LeNet with batch normalization
    net_bn.train()
    running_loss_bn = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer_bn.zero_grad()
        outputs = net_bn(inputs)
        loss_bn = criterion(outputs, labels)
        loss_bn.backward()
        optimizer_bn.step()
        running_loss_bn += loss_bn.item()
    #print('[%d, %5d] loss_bn: %.3f' % (epoch + 1, i + 1, running_loss_bn / len(trainloader)))

    # Train LeNet without batch normalization
    net.train()
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    #print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / len(trainloader)))

    # Test models on validation set
    net.eval()
    net_bn.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Epoch %d: Accuracy of the network without batch normalization on the test images: %d %%' % (epoch + 1, 100 * correct / total))

    net_bn.eval()
    correct_bn, total_bn = 0, 0
    with torch.no_grad():
      for data in testloader:
          images, labels = data[0].to(device), data[1].to(device)
          outputs_bn = net_bn(images)
          _, predicted_bn = torch.max(outputs_bn.data, 1)
          total_bn += labels.size(0)
          correct_bn += (predicted_bn == labels).sum().item()

    print('Epoch %d: Accuracy of the network with batch normalization on the test images: %d %%' % (epoch + 1, 100 * correct_bn / total_bn))

Files already downloaded and verified
Files already downloaded and verified
Epoch 1: Accuracy of the network without batch normalization on the test images: 37 %
Epoch 1: Accuracy of the network with batch normalization on the test images: 50 %
Epoch 2: Accuracy of the network without batch normalization on the test images: 48 %
Epoch 2: Accuracy of the network with batch normalization on the test images: 57 %
Epoch 3: Accuracy of the network without batch normalization on the test images: 53 %
Epoch 3: Accuracy of the network with batch normalization on the test images: 59 %
Epoch 4: Accuracy of the network without batch normalization on the test images: 54 %
Epoch 4: Accuracy of the network with batch normalization on the test images: 63 %
Epoch 5: Accuracy of the network without batch normalization on the test images: 58 %
Epoch 5: Accuracy of the network with batch normalization on the test images: 64 %
Epoch 6: Accuracy of the network without batch normalization on the test images

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

# Define LeNet with batch normalization
class LeNetBN(nn.Module):
    def __init__(self):
        super(LeNetBN, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.bn1 = nn.BatchNorm2d(6)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.bn2 = nn.BatchNorm2d(16)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.bn3 = nn.BatchNorm1d(120)
        self.fc2 = nn.Linear(120, 84)
        self.bn4 = nn.BatchNorm1d(84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool1(torch.relu(self.bn1(self.conv1(x))))
        x = self.pool2(torch.relu(self.bn2(self.conv2(x))))
        x = x.view(-1, 16*5*5)
        x = torch.relu(self.bn3(self.fc1(x)))
        x = torch.relu(self.bn4(self.fc2(x)))
        x = self.fc3(x)
        return x

# Define LeNet without batch normalization
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool1(torch.relu(self.conv1(x)))
        x = self.pool2(torch.relu(self.conv2(x)))
        x = x.view(-1, 16*5*5)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Define data transforms
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.2010])
])

# Load CIFAR-10 dataset
trainset = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
testset = datasets.FashionMNIST(root='./data', train=False, download=True,transform=transform )
# Load CIFAR-10 dataset
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
testset = datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

# Define data loaders
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=False, num_workers=2)

# Define device to train on
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Initialize LeNet models
net_bn = LeNetBN().to(device)
net = LeNet().to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer_bn = optim.SGD(net_bn.parameters(), lr=1.5, momentum=0.9)
optimizer = optim.SGD(net.parameters(), lr=1.5, momentum=0.9)

# Train models for 10 epochs
for epoch in range(10):

    # Train LeNet with batch normalization
    net_bn.train()
    running_loss_bn = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer_bn.zero_grad()
        outputs = net_bn(inputs)
        loss_bn = criterion(outputs, labels)
        loss_bn.backward()
        optimizer_bn.step()
        running_loss_bn += loss_bn.item()
    #print('[%d, %5d] loss_bn: %.3f' % (epoch + 1, i + 1, running_loss_bn / len(trainloader)))

    # Train LeNet without batch normalization
    net.train()
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    #print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / len(trainloader)))

    # Test models on validation set
    net.eval()
    net_bn.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Epoch %d: Accuracy of the network without batch normalization on the test images: %d %%' % (epoch + 1, 100 * correct / total))

    net_bn.eval()
    correct_bn, total_bn = 0, 0
    with torch.no_grad():
      for data in testloader:
          images, labels = data[0].to(device), data[1].to(device)
          outputs_bn = net_bn(images)
          _, predicted_bn = torch.max(outputs_bn.data, 1)
          total_bn += labels.size(0)
          correct_bn += (predicted_bn == labels).sum().item()

    print('Epoch %d: Accuracy of the network with batch normalization on the test images: %d %%' % (epoch + 1, 100 * correct_bn / total_bn))

Files already downloaded and verified
Files already downloaded and verified
Epoch 1: Accuracy of the network without batch normalization on the test images: 10 %
Epoch 1: Accuracy of the network with batch normalization on the test images: 27 %
Epoch 2: Accuracy of the network without batch normalization on the test images: 10 %
Epoch 2: Accuracy of the network with batch normalization on the test images: 12 %
Epoch 3: Accuracy of the network without batch normalization on the test images: 10 %
Epoch 3: Accuracy of the network with batch normalization on the test images: 10 %
Epoch 4: Accuracy of the network without batch normalization on the test images: 10 %
Epoch 4: Accuracy of the network with batch normalization on the test images: 10 %
Epoch 5: Accuracy of the network without batch normalization on the test images: 10 %
Epoch 5: Accuracy of the network with batch normalization on the test images: 10 %
Epoch 6: Accuracy of the network without batch normalization on the test images